In [22]:
import pandas as pd
import numpy as np

In [23]:
import pickle

with open("stock_pickle.pkl","rb") as f:
    X_train_scaled, X_test_scaled, y_train, y_test = pickle.load(f)

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [25]:
timesteps = 10

def create_sequence(X, y, timesteps=10):
    X_seq, y_seq = [], []
    for i in range(timesteps, len(X)):
        X_seq.append(X[i-timesteps:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

X_train_seq, y_train_seq = create_sequence(X_train_scaled, y_train, timesteps)
X_test_seq, y_test_seq = create_sequence(X_test_scaled, y_test, timesteps)

print("X_train_seq shape:", X_train_seq.shape) 
print("y_train_seq shape:", y_train_seq.shape)

X_train_seq shape: (547, 10, 6)
y_train_seq shape: (547,)


C:\Users\Bijay\AppData\Local\Temp\ipykernel_25328\1456522566.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_seq.append(y[i])


In [26]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(1))

c:\Users\Bijay\Downloads\DJANGO_PROJECTS\ML\codveda\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [27]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

In [28]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 10, 50)         │        11,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 10, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,651 (123.64 KB)

 Trainable params: 31,651 (123.64 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
from sklearn.preprocessing import MinMaxScaler

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train_seq.reshape(-1,1))
y_test_scaled  = scaler_y.transform(y_test_seq.reshape(-1,1))

In [36]:
model.fit(
    X_train_seq, y_train_scaled,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_seq, y_test_scaled),
    verbose=1
)

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1848 - val_mse: 0.1848
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0021 - mse: 0.0021 - val_loss: 0.1845 - val_mse: 0.1845
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1902 - val_mse: 0.1902
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.1713 - val_mse: 0.1713
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0021 - mse: 0.0021 - val_loss: 0.1739 - val_mse: 0.1739
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.1931 - val_mse: 0.1931
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.1667 - val_mse: 0.1667
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.2062 - val_mse: 0.2062
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.001

In [37]:
y_pred_scaled = model.predict(X_test_seq)
y_pred = scaler_y.inverse_transform(y_pred_scaled).ravel() 

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [38]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test_seq, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_seq, y_pred))
r2 = r2_score(y_test_seq, y_pred)

print("LSTM Results:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R2 Score:", r2)

LSTM Results:
MAE: 15.136252260843913
RMSE: 17.619494723674517
R2 Score: -0.7911534542755438


In [39]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

(557, 6)
(250, 6)
(557,)
(250,)
